## Sandbox with model

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import torch


from semeval.experiments.kosenko.language_bind.LanguageBind.languagebind import (
    LanguageBind,
    to_device,
    transform_dict,
    LanguageBindImageTokenizer,
)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 

In [2]:
device = "cuda:0"
device = torch.device(device)
clip_type = {
    "video": "LanguageBind_Video_FT",  # also LanguageBind_Video
    # "audio": "LanguageBind_Audio_FT",  # also LanguageBind_Audio
    # "thermal": "LanguageBind_Thermal",
    # "image": "LanguageBind_Image",
    # "depth": "LanguageBind_Depth",
}

languagebind_model = LanguageBind(clip_type=clip_type, cache_dir="/code/cache_dir")
languagebind_model = languagebind_model.to(device)
# model.eval()
pretrained_ckpt = f"LanguageBind/LanguageBind_Image"
tokenizer = LanguageBindImageTokenizer.from_pretrained(
    pretrained_ckpt, cache_dir="/code/cache_dir/tokenizer_cache_dir"
)
modality_transform = {
    c: transform_dict[c](languagebind_model.modality_config[c])
    for c in clip_type.keys()
}

### example of classifier

In [3]:
video = [
    "semeval/experiments/kosenko/language_bind/LanguageBind/assets/video/0.mp4",
    "semeval/experiments/kosenko/language_bind/LanguageBind/assets/video/0.mp4",
]
language = [
    "Two pandas are eating bamboo.",
    "Two pandas are eating bamboo.",
]

inputs = {
    "video": to_device(modality_transform["video"](video), device),
}
inputs["language"] = to_device(
    tokenizer(
        language,
        max_length=77,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    ),
    device,
)


class VideoTextClassif(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = languagebind_model
        self.linear = torch.nn.Linear(768 * 2, 2, bias=False)

    def forward(self, x):
        result = self.model(x)
        # print(result)
        features = torch.cat(
            [
                result["video"],
                result["language"],
            ],
            dim=-1,
        )
        result = self.linear(features)
        return result


text_video_classif = VideoTextClassif()
text_video_classif.to(device)
output = text_video_classif(inputs)
loss_func = torch.nn.CrossEntropyLoss()
loss = loss_func(output, torch.tensor([1, 0], device=device))
print(loss)
loss.backward()

tensor(0.7536, device='cuda:0', grad_fn=<NllLossBackward0>)


### Эксперимент 1

Классификатор на основе текста и видео. На вход подается независимая реплика диалога и соответствующее видео к нему. Никакой следующий контекст диалога не используется.

На основе этого нужно предсказать эмоцию, то есть класс.

In [1]:
import numpy as np
import random


def random_seed(seed=42, rank=0):
    torch.manual_seed(seed + rank)
    np.random.seed(seed + rank)
    random.seed(seed + rank)


from datasets import load_dataset
from torchvision.io import read_video
import json
import torch
import os
from torch.utils.data import Dataset, DataLoader
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


dataset_path = "./SemEval-2024_Task3/training_data/Subtask_2_train.json"


dataset = json.loads(open(dataset_path).read())
print(len(dataset))


# dataset[0]


all_conversations = []


for item in dataset:
    all_conversations.extend(item["conversation"])
print(len(all_conversations))


# all_emotions = set([])


# for item in all_conversations:


#     all_emotions.update([item["emotion"]])
# for item in all_conversations:
#     print(item['video_name'])


# print(all_emotions)


all_emotions = [
    "surprise",
    "fear",
    "sadness",
    "neutral",
    "joy",
    "anger",
    "disgust",
]


emotions2labels = {em: i for i, em in enumerate(all_emotions)}


labels2emotions = {i: em for i, em in enumerate(all_emotions)}


print(emotions2labels)


print(labels2emotions)


training_data, test_data = train_test_split(all_conversations, test_size=0.04)


class ConversationsDataset(Dataset):
    def __init__(
        self,
        conversations,
        base_video_path="/code/SemEval-2024_Task3/training_data/train",
    ):
        self.conversations = conversations

        self.base_video_path = base_video_path

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        turn = self.conversations[idx]

        video_path = turn["video_name"]

        turn["video_name"] = f"{self.base_video_path}/{video_path}"

        turn["label"] = emotions2labels[turn["emotion"]]

        return turn


training_data = ConversationsDataset(conversations=training_data)
test_data = ConversationsDataset(conversations=test_data)


train_dataloader = DataLoader(training_data, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=2, shuffle=False)


next(iter(train_dataloader))

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1374
13619
{'surprise': 0, 'fear': 1, 'sadness': 2, 'neutral': 3, 'joy': 4, 'anger': 5, 'disgust': 6}
{0: 'surprise', 1: 'fear', 2: 'sadness', 3: 'neutral', 4: 'joy', 5: 'anger', 6: 'disgust'}


{'utterance_ID': tensor([ 2, 13]),
 'text': ['Oh , I can not tell you how great it was to look at the crowd and see your face !',
  'Hey .'],
 'speaker': ['Ross', 'Rachel'],
 'emotion': ['joy', 'neutral'],
 'video_name': ['/code/SemEval-2024_Task3/training_data/train/dia1368utt2.mp4',
  '/code/SemEval-2024_Task3/training_data/train/dia147utt13.mp4'],
 'label': tensor([4, 3])}

In [8]:
len(all_conversations), len(training_data), len(test_data)

(13619, 13074, 545)

In [33]:
list(next(iter(train_dataloader)).keys())

['utterance_ID', 'text', 'speaker', 'emotion', 'video_name', 'label']

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import torch
from transformers.modeling_outputs import TokenClassifierOutput

from semeval.experiments.kosenko.language_bind.LanguageBind.languagebind import (
    LanguageBind,
    to_device,
    transform_dict,
    LanguageBindImageTokenizer,
)


class VideoTextClassif(torch.nn.Module):
    def __init__(self, labels=2, clip_type=None):
        super().__init__()
        self.model = LanguageBind(
            clip_type=clip_type,
            cache_dir="/code/cache_dir",
        )
        self.linear = torch.nn.Linear(
            768 * 2,
            labels,
            bias=False,
        )

    def forward(self, x):
        result = self.model(x)
        # print(result)
        features = torch.cat(
            [
                result["video"],
                result["language"],
            ],
            dim=-1,
        )
        result = self.linear(features)
        return result


device = "cuda:0"
device = torch.device(device)
clip_type = {
    "video": "LanguageBind_Video_FT",
}
text_video_classif = VideoTextClassif(
    labels=len(all_emotions),
    clip_type=clip_type,
)
text_video_classif = text_video_classif.to(device)
pretrained_ckpt = f"LanguageBind/LanguageBind_Image"
tokenizer = LanguageBindImageTokenizer.from_pretrained(
    pretrained_ckpt, cache_dir="/code/cache_dir/tokenizer_cache_dir"
)
modality_transform = {
    c: transform_dict[c](text_video_classif.model.modality_config[c])
    for c in clip_type.keys()
}

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/code/semeval/experiments/kosenko/language_bind/L

In [3]:
text_video_classif.eval()
batch = next(iter(train_dataloader))
inputs = {
    "video": to_device(
        modality_transform["video"](batch["video_name"]),
        device,
    ),
    "language": to_device(
        tokenizer(
            batch["text"],
            max_length=77,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        ),
        device,
    ),
}

result = text_video_classif(inputs)
predicted_labels = result.argmax(-1).cpu().numpy()
test_f1_score = f1_score(
    batch["label"].numpy(),
    predicted_labels,
    average="macro",
)
print(test_f1_score)

0.0


In [4]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    text_video_classif.parameters(),
    lr=0.00001,
)
loss = loss_func(
    torch.tensor(
        [
            [0.45, 0.23],
            [0.55, 0.33],
        ],
        device=device,
    ),
    torch.tensor([1, 0], device=device),
)
loss

tensor(0.6992, device='cuda:0')

#### super simple train loop

In [5]:
epochs = 1

max_train_steps = 100

for epoch in range(epochs):
    for num_step, batch in tqdm.tqdm(enumerate(train_dataloader)):
        optimizer.zero_grad()
        # print(batch)
        inputs = {
            "video": to_device(
                modality_transform["video"](batch["video_name"]), device
            ),
            "language": to_device(
                tokenizer(
                    batch["text"],
                    max_length=77,
                    padding="max_length",
                    truncation=True,
                    return_tensors="pt",
                ),
                device,
            ),
        }

        result = text_video_classif(inputs)
        label = batch["label"].to(device)
        loss = loss_func(result, label)
        print(num_step, loss.item())
        loss.backward()
        optimizer.step()

        if num_step > max_train_steps:
            break

    # for
    #     # break

0it [00:00, ?it/s]

0 2.4116463661193848


1it [00:01,  1.19s/it]

1 2.118516206741333


2it [00:02,  1.01it/s]

...

100it [01:19,  1.31it/s]

100 2.2770862579345703


101it [01:19,  1.31it/s]

101 1.2192869186401367


101it [01:20,  1.25it/s]


### huggingface trainer (падает по памяти)

In [12]:
# import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# import torch
# from transformers.modeling_outputs import TokenClassifierOutput

# from semeval.experiments.kosenko.language_bind.LanguageBind.languagebind import (
#     LanguageBind,
#     to_device,
#     transform_dict,
#     LanguageBindImageTokenizer,
# )
# from typing import Dict, List, Optional
# from torch import nn
# from torch.utils.data import Dataset
# from transformers import Trainer
# from transformers import TrainingArguments
# import datasets

# import numpy as np
# import random


# def random_seed(seed=42, rank=0):
#     torch.manual_seed(seed + rank)
#     np.random.seed(seed + rank)
#     random.seed(seed + rank)


# from datasets import load_dataset
# from torchvision.io import read_video
# import json
# import torch
# import os
# from torch.utils.data import Dataset, DataLoader
# import tqdm
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score


# class CustomTrainer(Trainer):
#     def compute_loss(
#         self,
#         model,
#         inputs,
#         return_outputs=False,
#     ):
#         video_paths = [
#             f"{base_path}/{video_path}"
#             for base_path, video_path in zip(
#                 inputs["video_base_path"], inputs["video_name"]
#             )
#         ]
#         custom_inputs = {
#             "video": to_device(
#                 modality_transform["video"](video_paths),
#                 device,
#             ),
#             "language": to_device(
#                 tokenizer(
#                     inputs["text"],
#                     max_length=77,
#                     padding="max_length",
#                     truncation=True,
#                     return_tensors="pt",
#                 ),
#                 device,
#             ),
#         }

#         # forward pass
#         outputs = model(custom_inputs)
#         label = inputs["label"].to(device)
#         loss_func = torch.nn.CrossEntropyLoss()
#         loss = loss_func(outputs, label)
#         torch.cuda.empty_cache()
#         return (loss, outputs) if return_outputs else loss

#     def get_train_dataloader(self):
#         train_dataset = self.train_dataset
#         return DataLoader(
#             train_dataset,
#             batch_size=self.args.per_device_train_batch_size,
#             shuffle=True,
#         )

#     def get_eval_dataloader(self, eval_dataset):
#         return DataLoader(
#             self.eval_dataset,
#             batch_size=self.args.per_device_eval_batch_size,
#             shuffle=False,
#         )

#     def prediction_step(
#         self,
#         model,
#         inputs,
#         prediction_loss_only,
#         ignore_keys=None,
#     ):
#         # print(inputs)
#         video_paths = [
#             f"{base_path}/{video_path}"
#             for base_path, video_path in zip(
#                 inputs["video_base_path"], inputs["video_name"]
#             )
#         ]
#         custom_inputs = {
#             "video": to_device(
#                 modality_transform["video"](video_paths),
#                 device,
#             ),
#             "language": to_device(
#                 tokenizer(
#                     inputs["text"],
#                     max_length=77,
#                     padding="max_length",
#                     truncation=True,
#                     return_tensors="pt",
#                 ),
#                 device,
#             ),
#         }

#         # forward pass
#         outputs = model(custom_inputs)
#         label = inputs["label"].to(device)
#         loss_func = torch.nn.CrossEntropyLoss()
#         loss = loss_func(outputs, label)
#         torch.cuda.empty_cache()
#         if prediction_loss_only:
#             return (loss, None, None)

#         return (loss, outputs, label)


# def compute_metrics(eval_preds):
#     # metric = evaluate.load("glue", "mrpc")
#     print(eval_preds)
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)
#     # return metric.compute(predictions=predictions, references=labels)
#     return 1.0


# class VideoTextClassif(torch.nn.Module):
#     def __init__(self, labels=2, clip_type=None):
#         super().__init__()
#         self.model = LanguageBind(
#             clip_type=clip_type,
#             cache_dir="/code/cache_dir",
#         )
#         self.linear = torch.nn.Linear(
#             768 * 2,
#             labels,
#             bias=False,
#         )

#     def forward(self, x):
#         result = self.model(x)
#         # print(result)
#         features = torch.cat(
#             [
#                 result["video"],
#                 result["language"],
#             ],
#             dim=-1,
#         )
#         result = self.linear(features)
#         return result


# class ConversationsDataset(Dataset):
#     def __init__(
#         self,
#         conversations,
#         base_video_path="/code/SemEval-2024_Task3/training_data/train",
#     ):
#         self.conversations = conversations

#         self.base_video_path = base_video_path

#     def __len__(self):
#         return len(self.conversations)

#     def __getitem__(self, idx):
#         turn = self.conversations[idx]

#         turn["video_name"] = turn["video_name"]
#         turn["video_base_path"] = self.base_video_path
#         # print(video_path)
#         turn["label"] = emotions2labels[turn["emotion"]]

#         return turn


# if __name__ == "__main__":
#     dataset_path = "./SemEval-2024_Task3/training_data/Subtask_2_train.json"

#     dataset = json.loads(open(dataset_path).read())
#     print(len(dataset))

#     # dataset[0]

#     all_conversations = []

#     for item in dataset:
#         all_conversations.extend(item["conversation"])
#     print(len(all_conversations))

#     all_emotions = [
#         "surprise",
#         "fear",
#         "sadness",
#         "neutral",
#         "joy",
#         "anger",
#         "disgust",
#     ]

#     emotions2labels = {em: i for i, em in enumerate(all_emotions)}

#     labels2emotions = {i: em for i, em in enumerate(all_emotions)}

#     print(emotions2labels)

#     print(labels2emotions)

#     training_data_list, test_data_list = train_test_split(
#         all_conversations, test_size=0.04
#     )
#     training_data_list = training_data_list[:10]
#     test_data_list = test_data_list[:10]
#     training_data = ConversationsDataset(conversations=training_data_list)
#     test_data = ConversationsDataset(conversations=test_data_list)

#     # train_dataloader = DataLoader(training_data, batch_size=2, shuffle=True)
#     # test_dataloader = DataLoader(test_data, batch_size=2, shuffle=False)

#     # next(iter(train_dataloader))
#     device = "cuda:0"
#     device = torch.device(device)
#     clip_type = {
#         "video": "LanguageBind_Video_FT",
#     }
#     text_video_classif = VideoTextClassif(
#         labels=len(all_emotions),
#         clip_type=clip_type,
#     )
#     text_video_classif = text_video_classif.to(device)
#     # text_video_classif.half()
#     pretrained_ckpt = f"LanguageBind/LanguageBind_Image"
#     tokenizer = LanguageBindImageTokenizer.from_pretrained(
#         pretrained_ckpt, cache_dir="/code/cache_dir/tokenizer_cache_dir"
#     )
#     modality_transform = {
#         c: transform_dict[c](text_video_classif.model.modality_config[c])
#         for c in clip_type.keys()
#     }

#     training_args = TrainingArguments(
#         output_dir="semeval/experiments/kosenko/language_bind/train_results/",
#         evaluation_strategy="epoch",
#         eval_steps=1,
#         report_to="none",
#         per_device_train_batch_size=1,
#         per_device_eval_batch_size=1,
#         gradient_accumulation_steps=4,
#         fp16=True,
#         remove_unused_columns=False,
#         label_names=[
#             "utterance_ID",
#             "text",
#             "speaker",
#             "emotion",
#             "video_name",
#             "label",
#         ],
#     )

#     # hf_training_data = datasets.Dataset.from_list([item for item in training_data])
#     # hf_test_data = datasets.Dataset.from_list([item for item in test_data])

#     trainer = CustomTrainer(
#         model=text_video_classif,
#         args=training_args,
#         # train_dataset=hf_training_data,
#         # eval_dataset=hf_test_data,
#         train_dataset=training_data,
#         eval_dataset=test_data,
#         compute_metrics=compute_metrics,
#     )
#     trainer.train()

{'labels': tensor([5, 5], device='cuda:0'), 'utterance_ID': tensor([5, 5], device='cuda:0')}


KeyError: 'video_name'

In [9]:
hf_training_data.column_names

['utterance_ID', 'text', 'speaker', 'emotion', 'video_name', 'label']